# Reinforcement Learning Programming - CSCN 8020
### Assignment 1
* Student Name: Reham Abuarqoub 
* Student ID: 9062922


# Problem 1 

## Pick-and-Place Robot as an MDP

We want to design a reinforcement learning setup for a robot arm that repeats a **pick-and-place task**.  
The robot must learn to move **fast** and **smoothly**, while controlling its motors directly and using feedback from positions and velocities.

---

## MDP Design

### 1. States
The state should describe both the robot and the task:
- **Robot kinematics**: joint positions (`q`) and joint velocities (`qdot`)  
- **Task context**: object pose (`x_obj`), goal pose (`x_goal`)  
- **Gripper status**: open/closed (`g`), and whether the object is held (`c`)  
- **Optional**: collision or distance-to-obstacle indicators  

**Example state vector:**  
`s = [q, qdot, x_obj, x_goal, g, c]`

---

### 2. Actions
The agent must control the motors directly:
- **Joint torque commands** (`tau ∈ R^n`, bounded by torque limits)  
- Alternative: joint velocity commands (if low-level controller is used)  

Torque control best matches the requirement and gives smoother motion.

---

### 3. Transition Dynamics
- Governed by robot physics (rigid-body dynamics with contacts).  
- Next state depends on the applied torque, current joint positions/velocities, and possible noise.  
- Gripper contact flag (`c`) switches on when the gripper closes around the object within tolerance.

---

### 4. Rewards
We want to encourage **success**, **speed**, **smoothness**, and **safety**:

- **+ Progress reward** → when the arm gets closer to the object (before grasp) or moves the object closer to the goal (after grasp).  
- **+ Success bonus** → large positive reward when the object is placed at the goal.  
- **− Time penalty** → small negative each step to encourage faster completion.  
- **− Energy penalty** → penalize large torques (`‖tau‖²`).  
- **− Smoothness penalty** → penalize jerky changes in velocity/torque.  
- **− Collision penalty** → strong negative if the robot collides or drops the object.  



### 5. Discount Factor (γ)
- Between **0.95 – 0.995**  
- Encourages both short-term speed and long-term smoothness.

---

### 6. Episode Termination
- **Success:** object placed at the goal and gripper released.  
- **Failure:** collision, dropped object, or time limit reached.  

---

## ✅ Summary
- **States**: include positions, velocities, object/goal info, and gripper state.  
- **Actions**: direct motor torques for fine and smooth control.  
- **Rewards**: balance success, speed, smoothness, and safety.  
- This MDP formulation allows reinforcement learning to train the robot arm for efficient and natural pick-and-place movements.


# Problem 2

## Two Iterations of Value Iteration

**Setup**  
- Grid layout:  
  `s1  s2`  
  `s3  s4`
- Rewards: `R(s1)=5, R(s2)=10, R(s3)=1, R(s4)=2`
- Discount: γ = 0.9  
- Transition: deterministic; if action hits wall → stay in same state.  
- Bellman backup:  
  \[
  V_{k+1}(s) = \max_a \big[ R(s) + \gamma V_k(s') \big]
  \]

---

### Iteration 1

1) **Initial value function \(V_0\):**

| State | V₀ |
|-------|----|
| s1    | 0  |
| s2    | 0  |
| s3    | 0  |
| s4    | 0  |

2) **Update step (future term is zero since V₀=0):**

| State | Calculation          | V₁ |
|-------|----------------------|----|
| s1    | 5 + 0 = 5            | 5  |
| s2    | 10 + 0 = 10          | 10 |
| s3    | 1 + 0 = 1            | 1  |
| s4    | 2 + 0 = 2            | 2  |

3) **Updated value function \(V_1\):**

| State | V₁ | Greedy action(s) |
|-------|----|------------------|
| s1    | 5  | any (all tie)    |
| s2    | 10 | any (all tie)    |
| s3    | 1  | any (all tie)    |
| s4    | 2  | any (all tie)    |

---

### Iteration 2

**Start with V₁ = {s1=5, s2=10, s3=1, s4=2}.**

| State | Actions considered | Calculation (max)                  | V₂  | Greedy action |
|-------|--------------------|-------------------------------------|-----|----------------|
| s1    | right→s2, down→s3, stay | max(5+0.9·10, 5+0.9·1, 5+0.9·5) = max(14, 5.9, 9.5) | 14  | right |
| s2    | left→s1, down→s4, stay | max(10+0.9·5, 10+0.9·2, 10+0.9·10) = max(14.5, 11.8, 19) | 19  | stay (up/right) |
| s3    | up→s1, right→s4, stay | max(1+0.9·5, 1+0.9·2, 1+0.9·1) = max(5.5, 2.8, 1.9) | 5.5 | up |
| s4    | up→s2, left→s3, stay  | max(2+0.9·10, 2+0.9·1, 2+0.9·2) = max(11, 2.9, 3.8) | 11  | up |

**Updated value function \(V_2\):**

| State | V₂  | Greedy action |
|-------|-----|---------------|
| s1    | 14  | right         |
| s2    | 19  | stay (up/right) |
| s3    | 5.5 | up            |
| s4    | 11  | up            |

---

### Takeaway
After two iterations, the value function already pushes the agent toward **s2** (the highest reward). The greedy policy shows clear direction:  
- From s1 → move **right** to s2  
- From s3 → move **up** to s1  
- From s4 → move **up** to s2  
- From s2 → best to **stay** (because it’s already the maximum reward spot).  


# Problem 3

## 5×5 Gridworld (Value Iteration)

**Environment**
- Grid size: 5×5. State = (row, col) with 0-based indexing.
- **Goal:** (4,4) — terminal/absorbing, reward **+10** on arrival.
- **Grey states:** {(0,4), (1,2), (3,0)} — reward **−5** on arrival.
- **All other states:** reward **−1** on arrival.
- **Actions:** Right, Left, Down, Up. If the move leaves the grid → stay in place.
- **Discount:** γ = 0.9.

**Algorithm (copy-based value iteration)**
- Keep two arrays: `V` (current values) and `V_new` (updates).
- For each state, compute the best one-step lookahead return using **only** `V`, write to `V_new`.
- After the sweep, set `V = V_new`.
- Stop when the largest change across states is below a small threshold `THETA`.

The script prints the optimal value function \(V^\*\) and the greedy policy \(\pi^\*\) as an arrow grid:
- ► ◄ ▼ ▲ for actions,
- **X** for grey cells,
- **G** for the goal.



### Step 1 — Reward as a List (Table)
We’ve already built `R[r,c]` as a single source-of-truth for rewards by state:
- Regular states = `−1`
- Grey states = `−5`
- Goal = `+10`

This satisfies “**reward function as a list**.”

### Step 2 — Copy-Based Value Iteration and Greedy Policy
We now implement **copy-based** value iteration:
- Build `V_new` from `V` without reusing updates within the sweep.
- After convergence, extract the greedy policy \( \pi^* \).


In [3]:

"""
Problem 3 — 5x5 Gridworld (stand-alone)

Environment:
- Deterministic transitions; invalid moves keep you in place.
- Reward paid on ARRIVAL to the next state s':
    +10 at the goal (4,4)  [terminal/absorbing]
    -5  at grey states {(0,4), (1,2), (3,0)}
    -1  otherwise
- Discount gamma = 0.9

Algorithm:
- Copy-based value iteration (write updates into V_new, then replace V ← V_new).
- Prints optimal V* and the greedy policy (arrows), with X for grey and G for goal.
"""

import numpy as np

# -----------------------------
# Configuration
# -----------------------------
N = 5
GAMMA = 0.9
THETA = 1e-6
MAX_ITERS = 10_000

GOAL  = (4, 4)
GREYS = {(0, 4), (1, 2), (3, 0)}  # match the figure in the prompt

# action index -> (Δrow, Δcol)
ACTIONS = {
    0: (0, 1),    # Right
    1: (0, -1),   # Left
    2: (1, 0),    # Down
    3: (-1, 0),   # Up
}
ARROW = {0: "►", 1: "◄", 2: "▼", 3: "▲"}

# -----------------------------
# Environment helpers
# -----------------------------
def in_bounds(r: int, c: int) -> bool:
    """True if (r, c) is inside the 5x5 grid."""
    return 0 <= r < N and 0 <= c < N

def step(state: tuple[int, int], a_idx: int):
    """
    Deterministic transition; wall -> stay.
    Returns: (next_state, reward_on_arrival, done)
    """
    if state == GOAL:
        return state, 0.0, True  # absorbing goal

    dr, dc = ACTIONS[a_idx]
    r, c   = state
    nr, nc = r + dr, c + dc
    if not in_bounds(nr, nc):
        nr, nc = r, c  # hit wall: stay

    s_next = (nr, nc)
    if s_next == GOAL:
        return s_next, 10.0, True
    if s_next in GREYS:
        return s_next, -5.0, False
    return s_next, -1.0, False

# -----------------------------
# Value Iteration (copy-based)
# -----------------------------
def value_iteration():
    """
    Copy-based value iteration:
      - Build V_new from the current V (no reuse within the sweep),
      - Replace V ← V_new after each sweep,
      - Stop when the max change is below THETA.
    Returns: (V*, number_of_sweeps)
    """
    V = np.zeros((N, N), dtype=float)  # V(GOAL)=0; the +10 is paid on arrival
    for it in range(MAX_ITERS):
        delta = 0.0
        V_new = V.copy()
        for r in range(N):
            for c in range(N):
                s = (r, c)
                if s == GOAL:
                    V_new[r, c] = 0.0
                    continue

                # One-step lookahead: max over actions of r + gamma * V(next)
                best_q = -1e18
                for a_idx in ACTIONS:
                    s_next, rwd, done = step(s, a_idx)
                    nr, nc = s_next
                    q = rwd + (0.0 if done else GAMMA * V[nr, nc])
                    if q > best_q:
                        best_q = q

                delta = max(delta, abs(best_q - V[r, c]))
                V_new[r, c] = best_q

        V = V_new
        if delta < THETA:
            return V, it + 1

    return V, MAX_ITERS

def greedy_policy_from_V(V: np.ndarray):
    """
    Build a printable grid of the greedy policy:
      ► ◄ ▼ ▲ for regular cells, X for greys, G for goal.
    """
    Pi = np.empty((N, N), dtype=object)
    for r in range(N):
        for c in range(N):
            s = (r, c)
            if s == GOAL:
                Pi[r, c] = "G"
                continue
            if s in GREYS:
                Pi[r, c] = "X"
                continue

            best_a, best_q = None, -1e18
            for a_idx in ACTIONS:
                s_next, rwd, done = step(s, a_idx)
                nr, nc = s_next
                q = rwd + (0.0 if done else GAMMA * V[nr, nc])
                if q > best_q:
                    best_q, best_a = q, a_idx

            Pi[r, c] = ARROW[best_a]
    return Pi

# -----------------------------
# Main
# -----------------------------
if __name__ == "__main__":
    V_star, iters = value_iteration()
    np.set_printoptions(precision=6, suppress=True)
    print(f"Optimal Value Function found in {iters} sweeps:")
    print(V_star)

    pi_star = greedy_policy_from_V(V_star)
    for row in pi_star:
        print(list(row))


Optimal Value Function found in 9 sweeps:
[[-0.434062  0.62882   1.8098    3.122     4.58    ]
 [ 0.62882   1.8098    3.122     4.58      6.2     ]
 [ 1.8098    3.122     4.58      6.2       8.      ]
 [ 3.122     4.58      6.2       8.       10.      ]
 [ 4.58      6.2       8.       10.        0.      ]]
['►', '►', '►', '▼', 'X']
['►', '▼', 'X', '►', '▼']
['►', '►', '►', '►', '▼']
['X', '►', '►', '►', '▼']
['►', '►', '►', '►', 'G']



- A convergence message like: `Converged in 9 sweeps.` (exact sweeps can vary slightly with tolerance).
- A nicely formatted **value table** where `G` marks the goal and `X` marks grey states.
- A matching **arrow policy** that moves toward `(4,4)` while avoiding/going around greys when optimal.



##  Task 2: In-Place Value Iteration (with comparison)

**What changes from Task 1?**  
- We keep **one** value table `V`.  
- When we back up a state `(r,c)`, we **write the new value directly into `V[r,c]`** and then **reuse** that fresh value for subsequent states in the same sweep.  
- This often reduces the number of sweeps to converge because each backup can benefit from the most recent neighbors.

**Goal.** Show that in-place value iteration converges to the **same** optimal value function \(V^\*\) and greedy policy \(\pi^\*\) as the copy-based method from Task 1, and briefly compare performance and complexity.

**Environment (same as Task 1).**  
- Grid 5×5, goal `(4,4)` with **+10** on arrival (terminal).  
- Grey cells `{(0,4), (1,2), (3,0)}` with **−5** on arrival.  
- All other arrivals **−1**.  
- Actions: Right, Left, Down, Up (deterministic; walls ⇒ stay).  
- Discount `γ = 0.9`.

**What to look at in the output.**  
- The printed `V* (in-place)` values and the arrow policy grid (► ◄ ▼ ▲; **X** grey; **G** goal).  
- A small summary that:  
  1) checks the **numerical equality** of `V*` from in-place vs copy-based,  
  2) reports **number of sweeps** and **wall-clock time** for both.  
*(There is no “number of episodes” here—value iteration is a planning algorithm, not Monte Carlo.)*


In [8]:
# Task 2: In-Place Value Iteration (stand-alone, includes a comparison vs copy-based)

import time
import numpy as np

# -----------------------------
# Config
# -----------------------------
N         = 5
GAMMA     = 0.9
THETA     = 1e-6
MAX_ITERS = 10_000

GOAL  = (4, 4)
GREYS = {(0, 4), (1, 2), (3, 0)}  # as in the figure

ACTIONS = {
    0: (0, 1),   # Right
    1: (0, -1),  # Left
    2: (1, 0),   # Down
    3: (-1, 0),  # Up
}
ARROW = {0: "►", 1: "◄", 2: "▼", 3: "▲"}

# -----------------------------
# Environment helpers
# -----------------------------
def in_bounds(r, c):
    return 0 <= r < N and 0 <= c < N

def step(state, a_idx):
    """Deterministic transition; walls keep you in place. Reward paid on ARRIVAL."""
    if state == GOAL:
        return state, 0.0, True
    dr, dc = ACTIONS[a_idx]
    r, c   = state
    nr, nc = r + dr, c + dc
    if not in_bounds(nr, nc):   # wall -> stay put
        nr, nc = r, c
    s_next = (nr, nc)
    if s_next == GOAL:  return s_next, 10.0, True
    if s_next in GREYS: return s_next, -5.0, False
    return s_next, -1.0, False

def policy_grid_from_V(V):
    """Pretty grid: ► ◄ ▼ ▲; X for grey; G for goal (greedy one-step lookahead)."""
    grid = []
    for r in range(N):
        row = []
        for c in range(N):
            s = (r, c)
            if s == GOAL:  row.append("G"); continue
            if s in GREYS: row.append("X"); continue
            best_a, best_q = None, -1e18
            for a_idx in ACTIONS:
                (nr, nc), rwd, done = *step(s, a_idx),
                q = rwd + (0.0 if done else GAMMA * V[nr, nc])
                if q > best_q:
                    best_q, best_a = q, a_idx
            row.append(ARROW[best_a])
        grid.append(row)
    return grid

# -----------------------------
# In-Place Value Iteration (this task)
# -----------------------------
def value_iteration_inplace():
    """
    In-place update:
      For each state in the sweep, compute the backup using the *current* V and
      immediately write V[r,c] = backup_value (then reuse it in the same sweep).
    """
    V = np.zeros((N, N), dtype=float)
    t0 = time.perf_counter()
    for it in range(MAX_ITERS):
        delta = 0.0
        for r in range(N):
            for c in range(N):
                s = (r, c)
                if s == GOAL:
                    V[r, c] = 0.0
                    continue
                v_old = V[r, c]
                best  = -1e18
                for a_idx in ACTIONS:
                    (nr, nc), rwd, done = *step(s, a_idx),
                    # uses up-to-date V inside this sweep
                    q = rwd + (0.0 if done else GAMMA * V[nr, nc])
                    if q > best:
                        best = q
                V[r, c] = best
                delta   = max(delta, abs(best - v_old))
        if delta < THETA:
            return V, it + 1, time.perf_counter() - t0
    return V, MAX_ITERS, time.perf_counter() - t0

# -----------------------------
# Copy-based Value Iteration (for comparison)
# -----------------------------
def value_iteration_copy_based():
    """
    Copy-based update:
      Build V_new from old V (no reuse within the sweep), then V <- V_new.
    """
    V = np.zeros((N, N), dtype=float)
    t0 = time.perf_counter()
    for it in range(MAX_ITERS):
        delta = 0.0
        V_new = V.copy()
        for r in range(N):
            for c in range(N):
                s = (r, c)
                if s == GOAL:
                    V_new[r, c] = 0.0
                    continue
                best = -1e18
                for a_idx in ACTIONS:
                    (nr, nc), rwd, done = *step(s, a_idx),
                    q = rwd + (0.0 if done else GAMMA * V[nr, nc])  # read old V
                    if q > best:
                        best = q
                delta = max(delta, abs(best - V[r, c]))
                V_new[r, c] = best
        V = V_new
        if delta < THETA:
            return V, it + 1, time.perf_counter() - t0
    return V, MAX_ITERS, time.perf_counter() - t0

# -----------------------------
# Run task + comparison
# -----------------------------
np.set_printoptions(precision=6, suppress=True)

V_inp, it_inp, t_inp = value_iteration_inplace()
print("=== In-Place Value Iteration (Task 2) ===")
print(f"Converged in {it_inp} sweeps, time={t_inp:.4f}s")
print("V* (in-place):\n", V_inp)
print("π* (► ◄ ▼ ▲; X=grey; G=goal):")
for row in policy_grid_from_V(V_inp):
    print(row)

# Optional: compare with copy-based to confirm same optimum
V_cpy, it_cpy, t_cpy = value_iteration_copy_based()
same = np.allclose(V_inp, V_cpy, atol=1e-10)
print("\n=== Comparison with copy-based VI (Task 1) ===")
print(f"Copy-based:  sweeps={it_cpy}, time={t_cpy:.4f}s")
print(f"In-place  :  sweeps={it_inp}, time={t_inp:.4f}s")
print("Do both methods reach the same V*? ", "YES" if same else "NO")

# Note: there is no concept of 'episodes' in value iteration (that applies to Monte Carlo).


=== In-Place Value Iteration (Task 2) ===
Converged in 9 sweeps, time=0.0031s
V* (in-place):
 [[-0.434062  0.62882   1.8098    3.122     4.58    ]
 [ 0.62882   1.8098    3.122     4.58      6.2     ]
 [ 1.8098    3.122     4.58      6.2       8.      ]
 [ 3.122     4.58      6.2       8.       10.      ]
 [ 4.58      6.2       8.       10.        0.      ]]
π* (► ◄ ▼ ▲; X=grey; G=goal):
['►', '►', '►', '▼', 'X']
['►', '▼', 'X', '►', '▼']
['►', '►', '►', '►', '▼']
['X', '►', '►', '►', '▼']
['►', '►', '►', '►', 'G']

=== Comparison with copy-based VI (Task 1) ===
Copy-based:  sweeps=9, time=0.0022s
In-place  :  sweeps=9, time=0.0031s
Do both methods reach the same V*?  YES


### Estimated Value Function \(V^*\) — 5×5 Gridworld

Environment:
- Goal (terminal): **(4,4)** with \(R=+10\) **on arrival**; \(V(\text{goal})=0\) (absorbing).
- Grey states: **(1,2), (3,0), (0,4)** with \(R=-5\) **on arrival**.
- Other states: \(R=-1\) **on arrival**.
- Actions: right, left, down, up; invalid moves keep you in place.
- Discount: \(gamma=0.9\).

Below is the **optimal state-value function \(V^*\)** for **every** state \((r,c)\), computed via value iteration (copy-based).  
Values are shown to **2 decimal places**. `G` marks the terminal goal; greys are included numerically since the assignment asks for *each state*.

| r\c |    0   |   1    |   2    |   3    |   4   |
|-----|:------:|:------:|:------:|:------:|:-----:|
| **0** | -0.43 |  0.63  |  1.81  |  3.12  |  4.58 |
| **1** |  0.63 |  1.81  |  3.12  |  4.58  |  6.20 |
| **2** |  1.81 |  3.12  |  4.58  |  6.20  |  8.00 |
| **3** |  3.12 |  4.58  |  6.20  |  8.00  | 10.00 |
| **4** |  4.58 |  6.20  |  8.00  | 10.00  |   G   |

**Notes**
- The goal state’s value is **0** because it is terminal; the **+10** is received upon **arrival** to it.
- Grey states still have **positive** optimal values due to proximity to the goal and discounting, even though arriving **into** them yields \(-5\).


In [6]:
# (Optional) Reproduce the exact V* numerically (copy-based value iteration)

import numpy as np

N = 5
GAMMA = 0.9
GOAL  = (4, 4)
GREYS = {(1, 2), (3, 0), (0, 4)}           # grey cells

ACTIONS = {0:(0,1), 1:(0,-1), 2:(1,0), 3:(-1,0)}  # right, left, down, up

def in_bounds(r, c): return 0 <= r < N and 0 <= c < N

# Reward table: -1 regular, -5 grey, +10 goal
R = np.full((N, N), -1.0)
for (r, c) in GREYS:
    R[r, c] = -5.0
R[GOAL] = +10.0

def step(state, a_idx):
    if state == GOAL:
        return state, 0.0, True
    dr, dc = ACTIONS[a_idx]
    r, c   = state
    nr, nc = r + dr, c + dc
    if not in_bounds(nr, nc):
        nr, nc = r, c
    s_next = (nr, nc)
    reward = R[nr, nc]              # reward on arrival
    done   = (s_next == GOAL)
    return s_next, reward, done

def value_iteration_copy(theta=1e-12, max_iters=100_000):
    V = np.zeros((N, N), dtype=float)
    for _ in range(max_iters):
        delta = 0.0
        V_new = V.copy()
        for r in range(N):
            for c in range(N):
                s = (r, c)
                if s == GOAL:
                    V_new[r, c] = 0.0
                    continue
                best_q = -1e18
                for a_idx in ACTIONS:
                    (nr, nc), rwd, done = step(s, a_idx)
                    q = rwd + (0.0 if done else GAMMA * V[nr, nc])
                    if q > best_q:
                        best_q = q
                delta = max(delta, abs(best_q - V[r, c]))
                V_new[r, c] = best_q
        V = V_new
        if delta < theta:
            return V
    return V

V_star = value_iteration_copy()
np.set_printoptions(precision=6, suppress=True)
print("V* (full precision array):\n", V_star)


V* (full precision array):
 [[-0.434062  0.62882   1.8098    3.122     4.58    ]
 [ 0.62882   1.8098    3.122     4.58      6.2     ]
 [ 1.8098    3.122     4.58      6.2       8.      ]
 [ 3.122     4.58      6.2       8.       10.      ]
 [ 4.58      6.2       8.       10.        0.      ]]


## Performance Comparison — Copy-based vs In-Place Value Iteration (human-readable)

| Aspect | **Copy-based Value Iteration** | **In-Place Value Iteration** |
|---|---|---|
| **What it does** | Computes a full new table `V_new` from the old `V` each sweep, then replaces `V ← V_new`. No reuse within the same sweep. | Updates `V` **immediately** cell-by-cell; later cells in the same sweep can use the freshest values (Gauss–Seidel style). |
| **Optimization time (wall-clock)** | Solid baseline. A bit of overhead from allocating/copying `V_new` each sweep; on small grids this is negligible, on large ones it adds up. | Often equal or **faster** because it can converge in fewer sweeps and avoids copying. Actual win depends on update order and cache friendliness. |
| **Sweeps to converge (typical behavior)** | Predictable but may take **more** sweeps because improvements don’t propagate until the next sweep. | Often **the same or fewer** sweeps since improvements ripple forward within the sweep. A good update order (e.g., sweeping from goal outward) helps. |
| **Number of episodes** | **N/A (0)** — This is Dynamic Programming using the model; it does not simulate trajectories. | **N/A (0)** — Same reason; no rollouts/episodes are generated. |
| **Time complexity** | \(O(\text{sweeps}\cdot|S|\cdot|A|)\). Each backup is \(O(|A|)\); total is backups × sweeps. | \(O(\text{sweeps}\cdot|S|\cdot|A|)\). Same per-backup cost; usually fewer sweeps in practice if the ordering is favorable. |
| **Memory footprint** | \(O(|S|)\) for `V` **plus** \(O(|S|)\) for `V_new` → effectively ~2× the value memory. | \(O(|S|)\) — only one value table in memory. Helpful on large state spaces. |
| **Parallelization / Vectorization** | **Easier** to parallelize a sweep (no data hazards because `V_new` reads from `V`). Plays nicely with GPUs/NumPy vectorization. | **Harder** to parallelize (write-after-read hazards). Often runs best as a single-threaded, cache-friendly loop. |
| **Sensitivity to update order** | **Low** — order inside a sweep doesn’t change the mathematics (only floating-point minutiae). | **High** — a smart ordering (e.g., from states near the goal outward) can noticeably reduce sweeps. A poor ordering can erase the advantage. |
| **Stability / Debuggability** | Very **transparent**: you can log `V` and `V_new` to see exactly what changed per sweep. | Slightly trickier to debug because values change in place; logging needs care to avoid mixing old/new states. |
| **Where it shines** | When you want clarity, easy theory, or to leverage parallel/vectorized hardware; when memory is not tight. | When memory is tight, or you want faster propagation with a good update order; often great on CPUs for large tabular MDPs. |
| **Gotchas / Pitfalls** | Extra memory/copy cost may matter on very large `|S|`. | Order dependence; harder to parallelize; if you pick a poor order, you might not see sweep reductions. |
| **Concrete example (this 5×5, γ=0.9)** | Converges in ~**9 sweeps**; wall-clock is tiny on a laptop (milliseconds). | Also ~**9 sweeps** with row-major order; could be **fewer** with a goal-centric order. Values and greedy policy **match** copy-based. |


# Problem 4

## Off-Policy Monte Carlo with Importance Sampling

**Setting (same gridworld as Problem 3)**  
- **Goal:** (4,4) → reward **+10** on arrival (terminal/absorbing)  
- **Grey cells:** {(0,4), (1,2), (3,0)} → reward **−5** on arrival  
- **Other cells:** reward **−1** on arrival  
- **Actions:** Right, Left, Down, Up (deterministic; walls ⇒ stay)  
- **Discount:** \(\gamma = 0.9\)

**Objective**  
Estimate the value function using **off-policy Monte Carlo (MC)** with **Weighted Importance Sampling (WIS)**:
- **Behavior policy** \(b\): uniform random over actions (used to generate episodes).
- **Target policy** \(\pi\): greedy w.r.t. current \(Q(s,a)\) (deterministic control).
- **Update (backward through each episode):**
  - Return \(G \leftarrow \gamma G + r\).
  - Cumulative weight \(W\) starts at 1; at each step multiply by \(\pi(a|s)/b(a|s)=4\) **only** if action = current greedy; otherwise stop (ratio 0).
  - Weighted-IS control update:
    \[
      C(s,a) \leftarrow C(s,a) + W,\qquad
      Q(s,a) \leftarrow Q(s,a) + \frac{W}{C(s,a)}\big(G - Q(s,a)\big)
    \]
- Report \(V_{\pi}(s)=\max_a Q(s,a)\) and the greedy policy grid.

**Comparison to Value Iteration (Problem 3)**  
Also compute \(V^*\) with copy-based Value Iteration for reference and report timing / episodes / a simple MAE between \(V_{\text{MC}}\) and \(V^*\).


In [8]:
# Off-policy Monte Carlo with Importance Sampling for 5x5 Gridworld
# Behavior b: uniform over 4 actions; Target π: greedy w.r.t learned Q
# Also compute reference V* by Value Iteration to compare

import numpy as np
import time

np.random.seed(42)

# -----------------------------
# Environment (same as Problem 3)
# -----------------------------
N = 5
GAMMA = 0.9
GOAL  = (4, 4)
GREYS = {(1, 2), (3, 0), (0, 4)}  # per spec

# Actions: 0=right, 1=left, 2=down, 3=up
ACTIONS = {0:(0,1), 1:(0,-1), 2:(1,0), 3:(-1,0)}
ARROW   = {0:"►", 1:"◄", 2:"▼", 3:"▲"}

def in_bounds(r, c): return 0 <= r < N and 0 <= c < N

# Reward table (state-based reward on arrival to s')
R = np.full((N, N), -1.0, dtype=float)          # regular
for (gr, gc) in GREYS: R[gr, gc] = -5.0         # greys
R[GOAL] = +10.0                                  # goal

def step(state, a_idx):
    """Deterministic transition; invalid -> stay. Reward on arrival."""
    if state == GOAL:
        return state, 0.0, True
    dr, dc = ACTIONS[a_idx]
    r, c   = state
    nr, nc = r + dr, c + dc
    if not in_bounds(nr, nc):
        nr, nc = r, c
    s_next = (nr, nc)
    reward = R[nr, nc]
    done   = (s_next == GOAL)
    return s_next, reward, done

# Pretty printers
def value_table_to_str(V, title="V table"):
    lines = [title]
    for r in range(N):
        row = []
        for c in range(N):
            if (r, c) == GOAL:
                row.append("  G  ")
            else:
                row.append(f"{V[r,c]:6.2f}")
        lines.append(" ".join(row))
    return "\n".join(lines)

def policy_to_str(Pi, title="Policy"):
    lines = [title]
    for r in range(N):
        lines.append(" ".join(f"{cell:>2}" for cell in Pi[r]))
    return "\n".join(lines)

def greedy_policy_from_V(V):
    Pi = np.empty((N, N), dtype=object)
    for r in range(N):
        for c in range(N):
            s = (r, c)
            if s == GOAL:
                Pi[r, c] = "G"
                continue
            if s in GREYS:
                Pi[r, c] = "X"
                continue
            best_a, best_q = None, -1e18
            for a_idx in ACTIONS:
                (nr, nc), rwd, done = step(s, a_idx)
                q = rwd + (0.0 if done else GAMMA * V[nr, nc])
                if q > best_q:
                    best_q, best_a = q, a_idx
            Pi[r, c] = ARROW[best_a]
    return Pi

# -----------------------------
# Value Iteration (reference)
# -----------------------------
def value_iteration_copy(theta=1e-9, max_iters=100_000):
    V = np.zeros((N, N), dtype=float)
    for it in range(max_iters):
        delta = 0.0
        V_new = V.copy()
        for r in range(N):
            for c in range(N):
                s = (r, c)
                if s == GOAL:
                    V_new[r, c] = 0.0
                    continue
                best_q = -1e18
                for a_idx in ACTIONS:
                    (nr, nc), rwd, done = step(s, a_idx)
                    q = rwd + (0.0 if done else GAMMA * V[nr, nc])
                    if q > best_q:
                        best_q = q
                delta = max(delta, abs(best_q - V[r, c]))
                V_new[r, c] = best_q
        V = V_new
        if delta < theta:
            return V, it + 1
    return V, max_iters

t0 = time.perf_counter()
V_star_ref, sweeps_ref = value_iteration_copy()
t1 = time.perf_counter()
Pi_star_ref = greedy_policy_from_V(V_star_ref)
vi_time_ms = (t1 - t0) * 1000.0

print(f"[Value Iteration] sweeps={sweeps_ref}, time={vi_time_ms:.2f} ms")
print(value_table_to_str(V_star_ref, "V* (Reference via Value Iteration)"))
print(policy_to_str(Pi_star_ref, "π* (Reference via Value Iteration)"))

# -----------------------------
# Off-policy MC Control (Weighted IS)
# -----------------------------
def generate_episode_behavior(max_steps=200):
    """Generate a trajectory under behavior b (uniform random over actions)."""
    # start from a random non-goal state
    while True:
        s0 = (np.random.randint(0, N), np.random.randint(0, N))
        if s0 != GOAL:
            break
    s = s0
    traj = []  # [(s,a,r), ...]
    for _ in range(max_steps):
        a = np.random.choice([0,1,2,3])  # behavior b uniform
        s_next, r, done = step(s, a)
        traj.append((s, a, r))
        s = s_next
        if done:
            break
    return traj

def offpolicy_mc_control_weighted_is(num_episodes=20000, max_steps=200):
    Q = np.zeros((N, N, 4), dtype=float)  # action-values
    C = np.zeros((N, N, 4), dtype=float)  # cumulative weights
    Pi = np.zeros((N, N), dtype=int)      # greedy target indices

    b_prob = 1.0 / 4.0  # behavior prob for each action

    for _ in range(num_episodes):
        ep = generate_episode_behavior(max_steps=max_steps)
        G = 0.0
        W = 1.0
        # process backward
        for (s, a, r) in reversed(ep):
            r_row, r_col = s
            G = GAMMA * G + r
            C[r_row, r_col, a] += W
            Q[r_row, r_col, a] += (W / C[r_row, r_col, a]) * (G - Q[r_row, r_col, a])

            # Improve π greedily
            Pi[r_row, r_col] = np.argmax(Q[r_row, r_col, :])

            # If the episode's action diverges from greedy at this (s), stop weighting earlier steps
            if a != Pi[r_row, r_col]:
                break

            # Update importance weight (π(a|s)=1 for greedy; b(a|s)=1/4)
            W *= (1.0 / b_prob)

            # Optional guard against blow-up
            if W > 1e12:
                break
    return Q, Pi

t2 = time.perf_counter()
Q_mc, Pi_mc_idx = offpolicy_mc_control_weighted_is(num_episodes=20000, max_steps=200)
t3 = time.perf_counter()
mc_time_ms = (t3 - t2) * 1000.0

# Derived V and arrows from MC
V_mc = np.max(Q_mc, axis=2)
Pi_mc_arrow = np.empty((N, N), dtype=object)
for r in range(N):
    for c in range(N):
        if (r, c) == GOAL:
            Pi_mc_arrow[r, c] = "G"
        elif (r, c) in GREYS:
            Pi_mc_arrow[r, c] = "X"
        else:
            Pi_mc_arrow[r, c] = ARROW[Pi_mc_idx[r, c]]

print("\n[Off-policy MC (Weighted IS)] episodes=20000, time={:.2f} ms".format(mc_time_ms))
print(value_table_to_str(V_mc, "V (Estimated via Off-policy MC IS)"))
print(policy_to_str(Pi_mc_arrow, "π (Greedy from learned Q; arrows)"))


[Value Iteration] sweeps=9, time=2.44 ms
V* (Reference via Value Iteration)
 -0.43   0.63   1.81   3.12   4.58
  0.63   1.81   3.12   4.58   6.20
  1.81   3.12   4.58   6.20   8.00
  3.12   4.58   6.20   8.00  10.00
  4.58   6.20   8.00  10.00   G  
π* (Reference via Value Iteration)
 ►  ►  ►  ▼  X
 ►  ▼  X  ►  ▼
 ►  ►  ►  ►  ▼
 X  ►  ►  ►  ▼
 ►  ►  ►  ►  G

[Off-policy MC (Weighted IS)] episodes=20000, time=48374.84 ms
V (Estimated via Off-policy MC IS)
 -0.43   0.62   1.73   3.08   4.53
  0.62   1.80   3.11   4.55   6.17
  1.80   3.11   4.55   6.18   7.99
  3.11   4.56   6.18   7.98  10.00
  4.40   6.17   7.98  10.00   G  
π (Greedy from learned Q; arrows)
 ►  ▼  ►  ▼  X
 ►  ▼  X  ▼  ▼
 ►  ►  ►  ▼  ▼
 X  ▼  ►  ▼  ▼
 ►  ►  ►  ►  G


### MC-IS vs Value Iteration — Comparison & Notes 

### Quick verdict
Both implementations are correct. Your **Value Iteration (VI)** provides the reference \(V^*\) and \(\pi^*\).  
Your **Off-policy MC with Weighted IS** learns a value function very close to VI and a nearly-greedy policy (some arrows differ at 20k episodes — expected with IS variance). 

---

### Side-by-side summary

| Method | Driver | “Iterations” | Wall-clock | Value vs VI | Policy vs VI | Comments |
|---|---|---:|---:|---|---|---|
| **Value Iteration (VI)** | Model-based DP | **9 sweeps** | **2.44 ms** | Baseline \(V^*\) | Baseline \(\pi^*\) | Deterministic backups; fastest when model is known. |
| **MC-IS (20k episodes)** | Off-policy episodes | **20,000 eps** | **48,374.84 ms** | **Very close** (diffs ~0.02–0.18 typical) | **A few cells differ** | High-variance by nature; needs more data to exactly match \(\pi^*\). |

**Examples of small value diffs (VI → MC):**  
- \((0,2)\): **1.81 → 1.73** (−0.08), \((0,3)\): **3.12 → 3.08** (−0.04), \((4,0)\): **4.58 → 4.40** (−0.18), \((3,3)\): **8.00 → 7.98** (−0.02).

**Policy arrows that differ (MC vs VI):** \((0,1), (1,3), (2,3), (3,1), (3,3)\) …(others match). This is normal at 20k episodes.

---

### Complexity & behavior

| Aspect | **Value Iteration (VI)** | **Off-policy MC + IS** |
|---|---|---|
| Needs model? | **Yes** (known transitions/rewards) | **No** (data only; uses behavior \(b\)) |
| Work unit | Sweeps over all states | Episodes (trajectories) from \(b\) |
| Time complexity | \(O(\text{sweeps}\cdot|S|\cdot|A|)\) | \(O(\text{episodes}\cdot \text{avg\_len})\) (constant-time update per step) |
| Memory | \(O(|S|)\) for \(V\) (+ small policy) | \(O(|S|\cdot|A|)\) for \(Q\) and cumulative weights \(C\) |
| Variance | Low (deterministic) | High (importance weights); **break** helps control variance |
| Convergence speed | Few sweeps here (9) | Slower; improves with more episodes / better \(b\) |

---

### Why MC differs slightly (and how to tighten it)

- **Finite-sample variance:** Weighted IS is noisy; with **20k** episodes you’re already close.  
- **Greedy “break” rule:** In off-policy control, you stop the backward pass once the action differs from current greedy \(\pi\). That means **early (far) states** get fewer weighted updates → slightly less accurate until episodes grow.  
- **Behavior coverage:** Uniform \(b\) is fine; a better-directed \(b\) (still covering all actions) or **more episodes (e.g., 50k–200k)** will make the MC policy fully match VI everywhere.

**If you want to show tighter agreement:**  
- Increase to **50k+ episodes** (you already saw that your lecture-agent at 50k matches the VI arrows).  
- Consider **per-decision importance sampling** (lower variance than trajectory-wise IS).  
- Keep the **weight cap/early break** to prevent blow-ups.

---


